### Assignment 3

This is Assignment 3. Deadline for submission is November, 16, 11pm EST. Please send your solutions in a gzipped archive to dmm2017@med.cornell.edu, with copy to chb4004@med.cornell.edu. No extensions will be granted. Your archive should contain this ipynb file with the code you wrote. However, if you are not comfortable with python, you can write in any other programming language, but provide instructions how to run/compile your code. Also, refer to the "Output data" section.

## De-Bruijn graph
The goal of this project is to build a basic De-Bruijn graph assembler.
Write a program that:
1. Takes fastq-file as input
2. Builds condensed De-Bruijn graph. Use k = 55.
3. Counts average k-mer coverage for each edge. (Kmer coverage is a number of times we see kmer in reads. Average kmer edge coverage is an average of all its kmer coverages).
4. Outputs result in a .dot file.  Each edge should have a label with it's length and average coverage.
5. Convert dot file into image file (png, svg, or jpg format).
6. Write TipClipping function that simplifies condensed De-Bruijn graph. Think about parameters and their values, so graph is properly simplified. Note, that after this procedure graph should remain condensed.
7. Produce dot and image files for simplified graph.
8. Measure running times of your solution for input files provided

## Hints

1. Add kmer and reverse-complement kmer to graph simultaneously, so your graph will be symmetric.

## Input data

You will be provided a three input fastq files of different sizes and reference genomes (so you could assess your results if needed).

## Output data

1. Short report on how you constructed graph, performed simplification, running times.
2. Figures that show assembly graph structure

## Grading

1. Correctness - graphs are correctly constructed, simplified, coverage is correct (even if it done only for the smallest sample) - 40%
2. Running time - 10% for each file (the largest file gives additional points). We don't set a hard time limit here, but your program should finish within minutes on your computer.
3. Report quality - 20%
4. Code quality - 20%


In [ ]:
from Bio import SeqIO
from graphviz import Digraph
import argparse
import pydot
import time
start_time = time.time()

class Vertex:

    def __init__(self, seq):
        self.seq = seq
        self.vertex_coverage = 1
        self.in_edges = {}
        self.out_edges = {}

    def increase_vertex_coverage(self):
        self.vertex_coverage += 1

class Edge:

    def __init__(self, current_kmer, next_kmer):
        self.seq = current_kmer + next_kmer[-1]
        self.in_vertices = {}
        self.out_vertices = {}


        self.edge_coverage = 0

    def increase_edge_coverage(self):
        self.edge_coverage += 1

    def calculation_edge_coverage(self, prev_vertex_cov, next_vertex_cov):
        self.edge_coverage = (prev_vertex_cov + next_vertex_cov)/2

    def merge_next(self, next):
        self.seq += next.seq[k:]
        self.edge_coverage = (self.edge_coverage * len(self.seq) + next.edge_coverage * len(next.seq)) / (len(self.seq) + len(next.seq))

    def merge_prev(self, prev):
        self.seq = prev.seq + self.seq[k:]
        self.edge_coverage = (self.edge_coverage * len(self.seq) + prev.edge_coverage * len(prev.seq)) / (len(self.seq) + len(prev.seq))

class Graph:

    def __init__(self, k):
        self.vertices = {}
        self.k = k

    def add_read(self, read):

        if len(read) < self.k:
            return

        # first k-mer
        kmer = read[:self.k]
        if kmer in self.vertices:
            self.vertices[kmer].increase_vertex_coverage()
        else:
            self.vertices[kmer] = Vertex(kmer)

        # next k-mer iterations:
        for i in range(1, len(read)-self.k+1, 1):
            next_kmer = read[i:i+self.k]
            if next_kmer in self.vertices:
                self.vertices[next_kmer].increase_vertex_coverage()
            else:
                self.vertices[next_kmer] = Vertex(next_kmer)

            # add new edge
            new_edge = Edge(kmer, next_kmer)
            # add vertices
            self.vertices[next_kmer].in_edges[kmer] = [new_edge]
            self.vertices[kmer].out_edges[next_kmer] = [new_edge]
            kmer = next_kmer

    def coverage_calculating(self):
        for current_vertex in self.vertices.keys():
            for next_vertex in self.vertices[current_vertex].out_edges.keys():
                self.vertices[current_vertex].out_edges[next_vertex][0] \
                    .calculation_edge_coverage(self.vertices[current_vertex].vertex_coverage,
                                               self.vertices[next_vertex].vertex_coverage)

    def merge_vertices(self, vertex, prev, next):
        self.vertices[prev].out_edges[next] = [Edge(prev, next)]
        self.vertices[next].in_edges[prev] = [Edge(prev, next)]
        self.vertices[prev].out_edges[next][0].merge_next(self.vertices[next].in_edges[prev][0])
        self.vertices[next].in_edges[prev][0].merge_prev(self.vertices[prev].out_edges[next][0])

        del self.vertices[vertex]
        del self.vertices[prev].out_edges[vertex]
        del self.vertices[next].in_edges[vertex]

    def compress(self):
        blacklist = []
        for vertex in self.vertices.keys():
            if (len(list(self.vertices[vertex].out_edges.keys()))) == 1 and (len(list(self.vertices[vertex].in_edges.keys())) == 1):
                blacklist.append(vertex)
        for vertex in blacklist:
            if vertex in self.vertices.keys() and len(self.vertices) > 2:
                prev, next = list(self.vertices[vertex].in_edges.keys())[0], list(self.vertices[vertex].out_edges.keys())[0]
                self.merge_vertices(vertex, prev, next)

    def cut(self):
        baseline = 1.2
        self.to_cut = [vertex for vertex in self.vertices.keys() if ((len(self.vertices[vertex].out_edges.keys()) == 0) and (len(self.vertices[vertex].in_edges.keys()) == 1))
                       or ((len(self.vertices[vertex].out_edges.keys()) == 1) and (len(self.vertices[vertex].in_edges.keys()) == 0))
                       or ((len(self.vertices[vertex].out_edges.keys()) == 0) and (len(self.vertices[vertex].in_edges.keys()) == 0))]
        for vertex in self.to_cut:
            if self.vertices[vertex].vertex_coverage <= baseline:
                if len(self.vertices[vertex].out_edges.keys()) == 1:
                    self.vertices[list(self.vertices[vertex].out_edges.keys())[0]].in_edges.pop(vertex)
                if len(self.vertices[vertex].in_edges.keys()) == 1:
                    self.vertices[list(self.vertices[vertex].in_edges.keys())[0]].out_edges.pop(vertex)

                del self.vertices[vertex]

    def graph_vis(self, result):


        dot = Digraph(comment='Assemble')


        if result == 'full':
            for k, v in self.vertices.items():

                dot.node(k, label=f'{k}')
                for kk, vv in v.out_edges.items():
                    dot.edge(k, kk, label=f'{vv[0].seq}')
        else:
            for k, v in self.vertices.items():

                dot.node(k, label=k)
                for kk, vv in v.out_edges.items():
                    print(v.out_edges)
                    dot.edge(k, kk, label=f'cov={vv[0].edge_coverage} len={len(vv[0].seq)}')

        #print(dot.source)
        dot.view()
        dot.save()



if __name__ == '__main__':

    parser = argparse.ArgumentParser(description='''Graph Visualization''')
    parser.add_argument('-i', '--input', help='Input fastq', type=str)
    parser.add_argument('-k', '--kmer', help='k-mer length', default=55, type=int)
    parser.add_argument('-t', '--type', help='full', default='full', type=str)
    parser.add_argument('-c', help='Graph Compression', action='store_true')



    args = parser.parse_args()

    i, k, t, s = args.input, args.kmer, args.type, args.strand

    my_graph = Graph(k)
    if s == 'fw':
        with open(i) as f:
            for record in SeqIO.parse(f, 'fastq'):
                my_graph.add_read(str(record.seq))
    else:
        with open(i) as f:
            for record in SeqIO.parse(f, 'fastq'):
                my_graph.add_read(str(record.reverse_complement().seq))

    my_graph.coverage_calculating()

    if args.c:
        my_graph.compress()
        my_graph.cut()

    my_graph.graph_vis(result=t)
    print("--- %s seconds ---" % (time.time() - start_time))


# Report

**Code Organization**

My code is divided into 3 classes:

1.   Vertex: This has graph vertices which represent individual k-mers. It has coincident edges and coverage 
2.   Edge: This contains edge coverage calculated as: 
edge coverage = (weighted edge coverage)*(number of combined edges)/ total number of combined edges. The condensing and tip clipping functions are also included in this segment. 
3. Graph: This contains dunctions to calculate coverage and visualize the graphs.

**Construction of the graph**

1. Parse and read the fastq files that are substrings of the reference genome
2. Each input string (kmer) of length k (here k = 55) was split it into overlapping substrings (kmers) of length k-1.  
3. Draw a directed multigraph between the k-1mers. Each edge on this graph represents the overlap between the k-1mers or a kmer to obtain the de bruijn graph.


**Condensing or simplification**

It is possible to simplify the graphs without any loss of information. When a vertex has only one incoming arc and one outgoing arc and if the outgoing arc is the only incoming arc on the next node, I merged the two nodes (function compress).

**Tip Clipping**

I further simplified the graph by deleting 'tips' or nodes that are connected only on one end. I set a baseline value of 2*k where k = kmer length. Every tip that is shorter than 2*k is removed. I chose this baseline value because a) this is longer than any short read and it is unlikely that any short read will have that many errors b)For longer reads, this is the value of two adjacent errors. Any length longer than this will only represent a real sequence or one that looks extremely real. 

When there are no more tips to remove, I condensed the graph once again. 

**Runtime** 

I ran most of my code from the commandline. The run time for the files were:

1k file:  

In [ ]:
python3 dbg2.py -i first1000.fastq -k 55 -t not -c
--- 0.27452516555786133 seconds ---

10k file:

In [ ]:
python3 dbg2.py -i first10000.fastq -k 55 -t not -c
--- 4.460830926895142 seconds ---

100k file

In [ ]:
python3 dbg2.py -i first10000.fastq -k 55 -t not -c
--- 86.75118064880371 seconds ---

**Improvements that can be made to the code**

My outputs are attached in the folder. I do think that my output for the last file is not correct, however I did not have the time to fix this. This algorithm also doesn't remove redundant paths or paths that start and end at the same node. This is another thing that could be improved. I tried my best to code more elegantly this time by use of functions and classes as opposed too many nested lists, to make sure the code runs within minutes. I have learned to do this better with every assignment. 